In [ ]:
import psycopg2 as p
from datetime import datetime

In [ ]:
# Name differentiation for review for fleets with 50 or more
# 12/17/2019 loop back to determine simpler way to compare first five letters of name
cur.execute("""
insert into mcmis_review
select *
from mcmis as m
join (select 
        fs.usdot, 
        m.census_num,
        fs.total_vehicles
      from fleetseek.fleet as fs
      join mcmis as m
      on fs.usdot = m.census_num 
      where upper(left(replace(replace(replace(replace(fs.company_name, '.', ''), '&', ''), '-', ''), ' ', ''), 5)) != upper(left(replace(replace(replace(m.name, '-', ''), '&', ''), ' ', ''), 5)) and 
      fs.total_vehicles > 49 and 
      fs.active_flag = 'Y'
      ) as sub_q
on m.census_num = sub_q.census_num
on conflict (mcmis_id) do nothing;

update mcmis_review
set review_reason = 'Company-Name'
where review_reason is null; 
""")

# All changes for fleets between 1 and 19 vehicles

### Fleet size changes for fleets between 1 and 19 vehicles

#### This needs to be the last update that happens

In [ ]:
#For any new updates on the Fleet table the suppress_redundant_updates_trigger will make sure that new n
cur.execute("""
update fleetseek.fleet
set 
    trucks_owned = sub_q.owntruck,
    tractors_owned = sub_q.owntract,
    trailers_owned = sub_q.trailers_owned,
    trucks_leased = sub_q.trucks_leased,
    tractors_leased = sub_q.tractors_leased,
    trailers_leased = sub_q.trailers_leased,
    total_trucks = sub_q.total_trucks,
    total_tractors = sub_q.total_tractors,
    total_trailers = sub_q.total_trailers,
    total_vehicles = sub_q.total_vehicles,
    total_vehicles_owned = sub_q.total_vehicles_owned,
    total_vehicles_leased = sub_q.total_vehicles_leased

from fleetseek.fleet as fs
join 
    (select 
        owntruck, 
        owntract, 
        owntrail,
        sum(trptruck+trmtruck) as trucks_leased,
        sum(trptract+trmtract) as tractors_leased,
        sum(trptrail+trmtrail) as trailers_leased,
        sum(owntruck+trptruck+trmtruck) as total_trucks, 
        sum(owntract+trmtract+trptract) as total_tractors,
        sum(owntrail+trmtrail+trptrail) as total_trailers,
        sum(owntruck+trptruck+trmtruck+owntract+trptract+trmtract) as total_vehicles,
        sum(owntruck+owntract) as total_vehicles_owned, 
        sum(trmtruck+trptruck+trmtract+trptract) as total_vehicles_leased,
        mcs150_date,
        census_num
     from mcmis
     group by owntruck, owntract, owntrail, mcs150_date, census_num
    ) as sub_q

on fs.usdot = sub_q.census_num 
where fs.update_date < sub_q.mcs150_date and
fs.active_flag = 'Y' and
fs.total_vehicles between 1 and 19)
""")
conn.commit()

## Drivers and mileage update

In [ ]:
cur.execute("""
update fleetseek.fleet
set 
    year_mileage =  m.mcs150mileageyear,
    vehicle_miles = m.mlg150,
    drivers = m.tot_drs,
    cdl_drivers = m.cdl_drs,
from mcmis as m
join fleetseek.fleet as fs
on m.census_num = fs.usdot
where fs.update_date < m.mcs150_date and
fs.total_vehicles between 1 and 19
""")
conn.commit()

## GVW Weight Class 8 update

In [ ]:
cur.execute("""
update fleetseek.fleet
set 
    gvw_class_8 = 'Y'
from fleetseek.fleet as fs
join (select 
        census_num, owntract, trmtract, trptract, mcs150_date
      from mcmis
      group by census_num, owntract, trmtract, trptract, mcs150_date
      having sum(owntract+trmtract+trptract) >= 1
     ) as sub_q

on fs.usdot = sub_q.census_num
where fs.update_date < sub_q.mcs150_date and
fs.total_vehicles between 1 and 19 and 
fs.gvw_class_8 != 'Y'
""")
conn.commit()

## Fleet trailer type 

In [ ]:
# Changes to the utility types but not for fleets with a hazmat indicator

cur.execute("""
update fleetseek.fleet_trailer_type
set trailer_type_id = 142
from fleetseek.fleet_trailer_type as ftt
join fleetseek.fleet as fs
on ftt.fleet_id = fs.fleet_id
join (select 
      m.census_num, m.owntract, m.trmtract, m.trptract, m.owntrail, m.trmtrail, m.trptrail, m.mcs150_date
      from mcmis as m
      group by m.census_num, m.owntract, m.trmtract, m.trptract, m.owntrail, m.trmtrail, m.trptrail, m.mcs150_date
      having (sum(owntract+trmtract+trptract) = 0 and sum(owntrail+trmtrail+trptrail) > 0) 
     limit 5) as sub_q
on fs.usdot = sub_q.census_num
where fs.update_date < sub_q.mcs150_date and
fs.total_vehicles between 1 and 19 and
fs.dot_hmind != 'Y'
""")
conn.commit()

## Private Fleet deactivation

In [ ]:
#For PFD companies that fall below a certain threshold of vehicles
cur.execute("""
update fleetseek.fleet
set active_flag = 'N'
from fleetseek.fleet as fs
where 
    fs.product_code = 'PFD' and
    fs.total_tractors = 1 and
    fs.total_trailers = 0 and
    fs.total_trucks <= 1 and 
    fs.total_vehicles <= 1 and
    fs.active_flag = 'Y'
""")
conn.commit()

## For-Hire to OO

#### Automatic update

In [ ]:
#This query will happen after all the deactivations and fleet size changes

cur.execute("""
update fleetseek.fleet as a
set 
    product_code = 'OOD',
    fleet_type_id = 105,
    sic_code = 4218
from fleetseek.fleet as fs
join (
    select 
        product_code, fleet_type_id, sic_code, total_vehicles, total_tractors, total_trailers, total_trucks, fleet_id
    from fleetseek.fleet
    where total_tractors <= 1 and
    total_trailers <= 1 and
    total_vehicles <= 1 and
    total_trucks = 0 and
    product_code not in  ('OOD', 'MMC', 'PFD') and
    active_flag = 'Y'
    ) as sub_q
on fs.fleet_id = sub_q.fleet_id
where a.product_code != 'OOD'
""")
conn.commit()

## OO Deactivation

#### Automatic update

In [ ]:
#Based on missing census_num in MCMIS
cur.execute("""
update fleetseek.fleet
set active_flag = 'N'
from fleetseek.fleet as fs
left join mcmis as m
on fs.usdot = m.census_num
where m.census_num is null and
fs.active_flag = 'Y' and 
fs.product_code = 'OOD'
""")
conn.commit()

## Deactivating For-Hire

#### Automatic update

In [ ]:
#For fleets between 1-24 power units based on missing census_num in MCMIS
cur.execute("""
update fleetseek.fleet
set active_flag = 'N'
from fleetseek.fleet as fs
left join mcmis as m
on fs.usdot = m.census_num
where 
    m.census_num is null and
    fs.active_flag = 'Y' and 
    fs.product_code = 'NMC' and
    fs.total_tractors between 1 and 24 and
    fs.usdot != 0
""")
conn.commit()

## MCMIS review For-Hire

#### Manual review

In [ ]:
#For-Hire fleets with 25+ power units and a missing DOT number in the MCMIS file will be reviewed by an analyst
#This will have to be a query that exports to sharepoint to be reviewed by an analyst
cur.execute("""
select 
    case when (fs.total_tractors >= 25 AND m.mcs150_date > now() - interval '30 months' and fs.product_code = 'NMC') then 'Exceeds MCS-150 Threshold'


select *
from fleetseek.fleet as fs
left join mcmis as m
on fs.usdot = m.census_num
where 
    m.census_num is null and
    fs.total_tractors >= 25 and
    fs.product_code = 'NMC'
""")

# 12/31/2019 review

#### Deactivations by analyst

In [1]:
import psycopg2 as p
from datetime import datetime
password = '********'

conn = p.connect(database = 'postgres', user = 'postgres', password = password, 
                       host = 'localhost', port = 5433)

cur = conn.cursor()

In [6]:
#For DOT's missing and for MCS-150 dates over thresholds
#Greater than 24 power units
password = 'Evergreen1'

conn = p.connect(database = 'postgres', user = 'postgres', password = password, 
                       host = 'localhost', port = 5433)
cur = conn.cursor()

# dd/mm/YY H:M:S
dt_string = datetime.now().strftime("%Y_%m_%d %H_%M_%S")


#New query as of 1/3/2020
query = """select *

from ( 
    select 
        fs.fleet_id, fs.product_code, fs.company_name, fs.website, fs.year_established, fs.revenue, fs.mcn, fs.usdot, fs.scac, fs.company_type_code, 
        fs.fleet_type_id, fs.safety_rating_type_id, fs.onsite_maintenance, fs.tl_ltl, fs.trucks_owned, fs.trucks_leased, 
        fs.tractors_owned, fs.tractors_leased, fs.trailers_owned, fs.trailers_leased, fs.gvw_class_12, fs.gvw_class_345, fs.gvw_class_6, fs.gvw_class_7,
        fs.gvw_class_8, fs.sic_code, fs.insurance_info_flag, fs.update_date, fs.active_flag, fs.calculated_revenue, fs.haz_trailers_owned, fs.haz_trailers_leased,
        fs.haz_trucks_owned, fs.haz_trucks_leased, fs.vehicle_miles, fs.drivers, fs.cdl_drivers, fs.average_miles, fs.gallonmin, fs.gallonmax, 
        fs.dot_hmind, fs.total_trucks, fs.total_tractors, fs.total_trailers, fs.total_vehicles, fs.total_vehicles_owned, fs.total_vehicles_leased,
        fs.last_called_date, fs.note as fs_note, ct.description as company_type_name, ct.type as company_type, ft.name as fleet_type, 
        case when pn1.phone_type_id = 1 then pn1.phone_number else null end as phone_number, case when pn2.phone_type_id = 2 then pn2.phone_number else null end as tf_number,
        case when pn3.phone_type_id = 3 then pn3.phone_number else null end as fax_number, l1.address1 as phy_address1, l1.address2 as phy_address2, l1.address3 as phy_address3,
        l1.city as phy_city, l1.county_fips as phy_county_fips, l1.state_code as phy_state_code, l1.zip_code as phy_zip_code, l1.country_code as phy_country_code,
        l1.latitude as phy_latitude, l1.longitude as phy_longitude, l2.address1 as mai_address1, l2.address2 as mai_address2, l2.address3 as mai_address3,
        l2.city as mai_city, l2.county_fips as mai_county_fips, l2.state_code as mai_state_code, l2.zip_code as mai_zip_code, l2.country_code as mai_country_code,
        l2.latitude as mai_latitude, l2.longitude as mai_longitude, '' as commodities, '' as trailer_types, fs.create_date, m.mcmis_id, m.act_stat, m.carship,
        m.census_num, m.name, m.name_dba, m.dbnum, m.phy_natn, m.reg, m.phy_str, m.phy_city, m.phy_cnty, m.phy_st, m.phy_zip, m.undeliv_phy, 
        m.tel_num, m.fax_num, m.mai_natn, m.mai_str, m.mai_city, m.mai_cnty, m.mai_st, m.mai_zip, m.undeliv_mai, m.iccdocket1, m.icc1, m.class, 
        m.classdef, m.crrinter, m.crrhmintra, m.crrintra, m.shpinter, m.shpintra, m.genfreight, m.household, m.metalsheet, m.motorveh,
        m.drivetow, m.logpole, m.bldgmat, m.mobilehome, m.machlrg, m.produce, m.oilfield, m.liqgas, m.intermodal, m.passengers, m.livestock,
        m.grainfeed, m.coalcoke, m.meat, m.garbage, m.usmail, m.chem, m.drybulk, m.coldfood, m.beverages, m.paperprod, m.utility,
        m.farmsupp, m.construct, m.waterwell, m.cargoothr, m.othercargo, m.hm_ind, m.owntruck, m.owntract, m.owntrail, m.trmtruck, 
        m.trmtract, m.trmtrail, m.trptruck, m.trptract, m.trptrail, m.tot_trucks, m.tot_pwr, m.interlt100, m.intergt100, m.inter_drs, m.intralt100,
        m.intragt100, m.intra_drs, m.avg_tld, m.tot_drs, m.cdl_drs, m.mlg150, m.miletot, m.mcs150_date, m.email_address, m.company_rep1, m.company_rep2,

        case 
            when (fs.total_tractors >= 25 AND m.mcs150_date < now() - interval '30 months' and fs.product_code = 'NMC' and fs.active_flag = 'Y' or 
            (fs.total_tractors >= 25 and m.mcs150_date < now() - interval '36 months' and fs.product_code = 'PFD' and fs.active_flag = 'Y')) then 'Exceeds MCS-150 Threshold'
        end as mcmis_review_reason,
        ' ' as note

    from fleetseek.fleet as fs
    join mcmis as m
    on fs.usdot = m.census_num
    join fleetseek.fleet_primary as fp
    on fs.fleet_id = fp.fleet_id
    left join fleetseek.phone_number as pn1
    on fp.primary_phone_id = pn1.phone_number_id
    left join fleetseek.phone_number as pn2
    on fp.primary_toll_free_id = pn2.phone_number_id
    left join fleetseek.phone_number as pn3
    on fp.primary_fax_id = pn3.phone_number_id
    left join fleetseek.location as l1
    on fp.primary_physical_location_id = l1.location_id
    left join fleetseek.location as l2 
    on fp.primary_mailing_location_id = l2.location_id
    left join fleetseek.company_type as ct
    on fs.company_type_code = ct.company_type_code
    left join fleetseek.fleet_type as ft
    on fs.fleet_type_id = ft.fleet_type_id
    ) as a

where 
    a.usdot != 0 and  
    a.mcmis_review_reason is not null and
    a.active_flag = 'Y'



union all



select *
from (
    select 
        fs.fleet_id, fs.product_code, fs.company_name, fs.website, fs.year_established, fs.revenue, fs.mcn, fs.usdot, fs.scac, fs.company_type_code, 
        fs.fleet_type_id, fs.safety_rating_type_id, fs.onsite_maintenance, fs.tl_ltl, fs.trucks_owned, fs.trucks_leased, 
        fs.tractors_owned, fs.tractors_leased, fs.trailers_owned, fs.trailers_leased, fs.gvw_class_12, fs.gvw_class_345, fs.gvw_class_6, fs.gvw_class_7,
        fs.gvw_class_8, fs.sic_code, fs.insurance_info_flag, fs.update_date, fs.active_flag, fs.calculated_revenue, fs.haz_trailers_owned, fs.haz_trailers_leased,
        fs.haz_trucks_owned, fs.haz_trucks_leased, fs.vehicle_miles, fs.drivers, fs.cdl_drivers, fs.average_miles, fs.gallonmin, fs.gallonmax, 
        fs.dot_hmind, fs.total_trucks, fs.total_tractors, fs.total_trailers, fs.total_vehicles, fs.total_vehicles_owned, fs.total_vehicles_leased,
        fs.last_called_date, fs.note as fs_note, ct.description as company_type_name, ct.type as company_type, ft.name as fleet_type, 
        case when pn1.phone_type_id = 1 then pn1.phone_number else null end as phone_number, case when pn2.phone_type_id = 2 then pn2.phone_number else null end as tf_number,
        case when pn3.phone_type_id = 3 then pn3.phone_number else null end as fax_number, l1.address1 as phy_address1, l1.address2 as phy_address2, l1.address3 as phy_address3,
        l1.city as phy_city, l1.county_fips as phy_county_fips, l1.state_code as phy_state_code, l1.zip_code as phy_zip_code, l1.country_code as phy_country_code,
        l1.latitude as phy_latitude, l1.longitude as phy_longitude, l2.address1 as mai_address1, l2.address2 as mai_address2, l2.address3 as mai_address3,
        l2.city as mai_city, l2.county_fips as mai_county_fips, l2.state_code as mai_state_code, l2.zip_code as mai_zip_code, l2.country_code as mai_country_code,
        l2.latitude as mai_latitude, l2.longitude as mai_longitude, '' as commodities, '' as trailer_types, fs.create_date, m.mcmis_id, m.act_stat, m.carship,
        m.census_num, m.name, m.name_dba, m.dbnum, m.phy_natn, m.reg, m.phy_str, m.phy_city, m.phy_cnty, m.phy_st, m.phy_zip, m.undeliv_phy, 
        m.tel_num, m.fax_num, m.mai_natn, m.mai_str, m.mai_city, m.mai_cnty, m.mai_st, m.mai_zip, m.undeliv_mai, m.iccdocket1, m.icc1, m.class, 
        m.classdef, m.crrinter, m.crrhmintra, m.crrintra, m.shpinter, m.shpintra, m.genfreight, m.household, m.metalsheet, m.motorveh,
        m.drivetow, m.logpole, m.bldgmat, m.mobilehome, m.machlrg, m.produce, m.oilfield, m.liqgas, m.intermodal, m.passengers, m.livestock,
        m.grainfeed, m.coalcoke, m.meat, m.garbage, m.usmail, m.chem, m.drybulk, m.coldfood, m.beverages, m.paperprod, m.utility,
        m.farmsupp, m.construct, m.waterwell, m.cargoothr, m.othercargo, m.hm_ind, m.owntruck, m.owntract, m.owntrail, m.trmtruck, 
        m.trmtract, m.trmtrail, m.trptruck, m.trptract, m.trptrail, m.tot_trucks, m.tot_pwr, m.interlt100, m.intergt100, m.inter_drs, m.intralt100,
        m.intragt100, m.intra_drs, m.avg_tld, m.tot_drs, m.cdl_drs, m.mlg150, m.miletot, m.mcs150_date, m.email_address, m.company_rep1, m.company_rep2,

        case 
            when (m.census_num is null and fs.total_tractors >= 25 and fs.product_code = 'NMC') then 'Inactive DOT' end as mcmis_review_reason,
        ' ' as note

    from fleetseek.fleet as fs
    join fleetseek.fleet_primary as fp
    on fs.fleet_id = fp.fleet_id
    left join fleetseek.phone_number as pn1
    on fp.primary_phone_id = pn1.phone_number_id
    left join fleetseek.phone_number as pn2
    on fp.primary_toll_free_id = pn2.phone_number_id
    left join fleetseek.phone_number as pn3
    on fp.primary_fax_id = pn3.phone_number_id
    left join fleetseek.location as l1
    on fp.primary_physical_location_id = l1.location_id
    left join fleetseek.location as l2 
    on fp.primary_mailing_location_id = l2.location_id
    left join fleetseek.company_type as ct
    on fs.company_type_code = ct.company_type_code
    left join fleetseek.fleet_type as ft
    on fs.fleet_type_id = ft.fleet_type_id
    left join mcmis as m
    on fs.usdot = m.census_num
    where m.census_num is null
    ) as b
where b.usdot != 0 and
b.mcmis_review_reason is not null and
b.active_flag = 'Y'
"""

#12/31/2019 query 
old_query = """select *

from ( select *,
        case 
            when (fs.total_tractors >= 25 AND m.mcs150_date < now() - interval '30 months' and fs.product_code = 'NMC' and fs.active_flag = 'Y' or 
            (fs.total_tractors >= 25 and m.mcs150_date < now() - interval '36 months' and fs.product_code = 'PFD' and fs.active_flag = 'Y')) then 'Exceeds MCS-150 Threshold'
            end as mcmis_review_reason,
        '' as note
        from fleetseek.fleet as fs
        join mcmis as m
        on fs.usdot = m.census_num) as a
where 
     a.usdot != 0 and  
     a.mcmis_review_reason is not null and
     a.active_flag = 'Y'




union all

select *
from (
    select *,
    case 
        when (m.census_num is null and fs.total_tractors >= 25 and fs.product_code = 'NMC') then 'Inactive DOT' end as mcmis_review_reason,
    '' as note
    from fleetseek.fleet as fs
    left join mcmis as m
    on fs.usdot = m.census_num
    where m.census_num is null) as b
where 
    b.usdot != 0 and
    b.mcmis_review_reason is not null and
    b.active_flag = 'Y'
limit 50
"""

outputquery = "COPY ({0}) TO STDOUT WITH CSV HEADER".format(query)


with open(r'\\informaplc.sharepoint.com@SSL\DavWWWRoot\sites\GAATLFS1\Shared Documents\FleetSeek Atlanta\ETL_updates\Update_output\Update_output' + dt_string + '.csv', 'w') as f:
    cur.copy_expert(outputquery, f)
    
    
    

## OO Deactivation

#### Automatic update

In [ ]:
#OO deactivated if MCS150 date is over 30 months old
cur.execute("""
update fleetseek.fleet
set active_flag = 'N'
select *
from fleetseek.fleet as fs
join mcmis as m
on fs.usdot = m.census_num
where 
    fs.active_flag = 'Y' and
    m.mcs150_date < now() - interval '30 months' and
    fs.product_code = 'OOD'
""")
conn.commit()

## FH Deactivation

#### Automatic update

In [ ]:
# FH deactivated if MCS150 date is over 30 months old and power units between 1 and 24
cur.execute("""
update fleetseek.fleet
set active_flag = 'N'
from fleetseek.fleet as fs
join mcmis as m
on fs.usdot = m.census_num
where
    fs.active_flag = 'Y' and
    m.mcs150_date < now() - interval '30 months' and
    fs.product_code = 'NMC' and
    fs.total_tractors between 1 and 24
""")

## FH Deactivation

#### Manual review

## OO Reinstate

#### Automatic update

In [ ]:
#12/19/2019 first pass. This process will likely need to be passed through Melissa data as well

cur.execute("""
update fleetseek.fleet
set 
    trucks_owned = sub_q.owntruck,
    tractors_owned = sub_q.owntract,
    trailers_owned = sub_q.owntrail,
    trucks_leased = sub_q.trucks_leased,
    tractors_leased = sub_q.tractors_leased,
    trailers_leased = sub_q.trailers_leased,
    total_trucks = sub_q.total_trucks,
    total_tractors = sub_q.total_tractors,
    total_trailers = sub_q.total_trailers,
    total_vehicles = sub_q.total_vehicles,
    total_vehicles_owned = sub_q.total_vehicles_owned,
    total_vehicles_leased = sub_q.total_vehicles_leased,
    dot_hmind = sub_q.hm_ind,
    year_mileage = sub_q.mcs150mileageyear,
    vehicle_miles = sub_q.mlg150,
    drivers = sub_q.tot_drs,
    cdl_drivers = sub_q.cdl_drivers,
    active_flag = 'Y'
from fleetseek.fleet as fs
join 
(
    select 
        census_num, name, owntruck, owntract, owntrail, sum(trptruck+trmtruck) as trucks_leased, sum(trptract+trmtract) as tractors_leased, 
        sum(trptrail+trmtrail) as trailers_leased, sum(owntruck+trptruck+trmtruck) as total_trucks, sum(owntract+trmtract+trptract) as total_tractors,
        sum(owntrail+trmtrail+trptrail) as total_trailers, sum(owntruck+trptruck+trmtruck+owntract+trptract+trmtract) as total_vehicles,
        sum(owntruck+owntract) as total_vehicles_owned, sum(trmtruck+trptruck+trmtract+trptract) as total_vehicles_leased, hm_ind, mcs150mileageyear, 
        mlg150, tot_drs, cdl_drs as cdl_drivers, mcs150_date, passengers, class, carship
    from mcmis
    group by census_num, name, owntruck, owntract, owntrail, hm_ind, mcs150mileageyear, mlg150, tot_drs, cdl_drs, mcs150_date, passengers, class, carship
) as sub_q
on fs.usdot = sub_q.census_num
where
    fs.active_flag = 'N' and
    fs.update_date < sub_q.mcs150_date and
    sub_q.total_trucks = 1 and 
    sub_q.total_tractors = 1 and
    sub_q.total_trailers <= 1 and
    sub_q.passengers is null and
    (sub_q.class similar to 'A%|B%' or
    sub_q.class = 'AB') and
    sub_q.carship like 'C%' and
    fs.product_code = 'OOD'
""")
conn.commit()

# Exporting a SQL query with Python

#### As of 12/31/2019 still need to export to SharePoint

#### As of 1/3/2020 able to export to Sharepoint 

In [15]:
pd.options.display.max_rows = 4000

In [11]:
test = pd.read_csv(r'\\informaplc.sharepoint.com@SSL\DavWWWRoot\sites\GAATLFS1\Shared Documents\FleetSeek Atlanta\ETL_updates\Update_output\Update_output2020_01_06 16_18_13.csv')
print(test.columns)

Index(['fleet_id', 'product_code', 'company_name', 'website',
       'year_established', 'revenue', 'mcn', 'usdot', 'scac',
       'company_type_code',
       ...
       'tot_drs', 'cdl_drs', 'mlg150', 'miletot', 'mcs150_date',
       'email_address', 'company_rep1', 'company_rep2', 'mcmis_review_reason',
       'note'],
      dtype='object', length=171)


In [12]:
columns = test.columns

In [ ]:
test.info(verbose = True)

In [4]:
# dd/mm/YY H:M:S
dt_string = datetime.now().strftime("%Y_%m_%d %H_%M_%S")

In [5]:
print("date and time =", dt_string)

date and time = 2020_01_02 11_32_45


In [29]:
import fnmatch

In [31]:
file_list = fnmatch.filter(os.listdir(r'C:\Users\YoungW\OneDrive - Informa plc\Documents\Google Analytics API project'), 'pageViews*.csv')

In [39]:
import glob
import os

list_of_files = glob.glob(r'C:\Users\YoungW\OneDrive - Informa plc\Documents\Google Analytics API project\*.csv') # * means all if need specific format then *.csv
oldest_file = min(list_of_files, key=os.path.getctime)
print(latest_file)

['C:\\Users\\YoungW\\OneDrive - Informa plc\\Documents\\Google Analytics API project\\2016-2019_data_JW_Final.csv', "C:\\Users\\YoungW\\OneDrive - Informa plc\\Documents\\Google Analytics API project\\pageViews January '16 - December '16.csv", "C:\\Users\\YoungW\\OneDrive - Informa plc\\Documents\\Google Analytics API project\\pageViews January '17 - December '17.csv", "C:\\Users\\YoungW\\OneDrive - Informa plc\\Documents\\Google Analytics API project\\pageViews January '18 - December '18.csv", "C:\\Users\\YoungW\\OneDrive - Informa plc\\Documents\\Google Analytics API project\\pageViews January '19 - November '19.csv", "C:\\Users\\YoungW\\OneDrive - Informa plc\\Documents\\Google Analytics API project\\sessions January '16 - November '19.csv"]
C:\Users\YoungW\OneDrive - Informa plc\Documents\Google Analytics API project\pageViews January '16 - December '16.csv


In [ ]:
with open(r'\\informaplc.sharepoint.com@SSL\DavWWWRoot\sites\GAATLFS1\Shared Documents\FleetSeek Atlanta\ETL_updates\Update_output\Update_output2020_01_02 11_32_45.txt', 'r+') as t:
    t.readlines()

In [52]:
t

<_io.TextIOWrapper name='\\\\informaplc.sharepoint.com@SSL\\DavWWWRoot\\sites\\GAATLFS1\\Shared Documents\\FleetSeek Atlanta\\ETL_updates\\Update_output\\Update_output2020_01_02 11_32_45.txt' mode='r+' encoding='cp1252'>

In [12]:
conn.close()

In [ ]:
import sys

#set up psycopg2 environment
import psycopg2

#driving_distance module
#note the lack of trailing semi-colon in the query string, as per the Postgres documentation
query = """
    select *
    from driving_distance ($$
        select
            gid as id,
            start_id::int4 as source,
            end_id::int4 as target,
            shape_leng::double precision as cost
        from network
        $$, %s, %s, %s, %s
    )
"""

#make connection between python and postgresql
conn = psycopg2.connect("dbname = 'routing_template' user = 'postgres' host = 'localhost' password = 'xxxx'")
cur = conn.cursor()

outputquery = "COPY ({0}) TO STDOUT WITH CSV HEADER".format(query)

with open('resultsfile', 'w') as f:
    cur.copy_expert(outputquery, f)

conn.close()